In [1]:
import os 
import ee 
import geemap
import time 
from glob import glob
from concurrent.futures import ThreadPoolExecutor
import geopandas as gpd 
import GoogleEarthEngineDatasets as geed 

geed.ee_api_initialize(api="high_volume")

cpus = int(os.cpu_count() * 0.75)
scale = 30#15 
pol = 'VV'
name = 'S1_VVVH'

data_dload_dir = f"/media/ljp238/12TBWolf/ARCHIEVE/GEEDataDownload/{name}" 
pathern = "/home/ljp238/Downloads/DRC_LiDAR_Roy2021/GEEdata/patches/*/*_tindex.gpkg"
gpkg_files = sorted(glob(pathern),reverse=True)
print(gpkg_files)

['/home/ljp238/Downloads/DRC_LiDAR_Roy2021/GEEdata/patches/S003E018/S003E018_tindex.gpkg', '/home/ljp238/Downloads/DRC_LiDAR_Roy2021/GEEdata/patches/S003E017/S003E017_tindex.gpkg', '/home/ljp238/Downloads/DRC_LiDAR_Roy2021/GEEdata/patches/S002E018/S002E018_tindex.gpkg', '/home/ljp238/Downloads/DRC_LiDAR_Roy2021/GEEdata/patches/S002E017/S002E017_tindex.gpkg']


In [22]:
from os.path import basename,join,isfile
def s1_median_image(aoi,pol='VH', opass='ASCENDING',idate='2019-01-01',fdate='2022-12-01'):
    collection = ee.ImageCollection("COPERNICUS/S1_GRD") \
        .filterBounds(aoi) \
        .filterDate('2022-01-01', '2022-01-31') \
        .filter(ee.Filter.eq('instrumentMode', 'IW')) \
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', pol))\
        .filter(ee.Filter.eq('resolution_meters',10)) \

    # Check if the collection is not empty
    size = collection.size().getInfo()
    if size == 0:
        print("No images found with the specified filters.")
    else:
        image = collection.median().clip(aoi)
        # Confirm available bands
        print(image.bandNames().getInfo())
        image = image.select(['VV', 'VH'])
        print(image.bandNames().getInfo())
        return image
    
def get_ee_geometry(i, g, name):
    ig = g.iloc[i:i+1,]
    bBox = [float(ig.minx), float(ig.miny), float(ig.maxx), float(ig.maxy)]
    fname = (basename(ig.location.values[0])).replace('..tif', '.tif')
    fname = fname.replace('.tif', f'_{name}.tif')
    region = ee.Geometry.Rectangle(bBox)
    return region, fname


def gee_download_geemap(image,outpath, scale):
    print(outpath)
    if isfile(outpath):
        print('Already downloaded') 
    else:
        geemap.ee_export_image(image, outpath, scale=scale)


def download_sentinel1(i,g,pol,name,S1tile_path,scale):
    region, fname = get_ee_geometry(i, g,name)
    s1img = s1_median_image(region, pol)
    outpath = join(S1tile_path, fname)
    gee_download_geemap(s1img,outpath, scale)

In [23]:
def sentinel1_download_par(i,g,pol,name,tname,S1tile_path,scale,gpkg_files):
    with ThreadPoolExecutor(cpus) as TEX:
        for j in range(g.shape[0]):
            if j > 2: break
            print(f'{j}/{g.shape[0]} @{tname} :: {i}{len(gpkg_files)}')
            TEX.submit(download_sentinel1,j,g,pol,name,S1tile_path,scale)

In [24]:
ti = time.perf_counter()
for i in range(len(gpkg_files)):
    if i > 0 : break
    gfile = gpkg_files[i]
    g = gpd.read_file(gfile)
    g[['minx','miny','maxx','maxy']] = g.bounds
    print('gfile', gfile)
    tname = os.path.basename(gfile).replace('.gpkg','')
    S1tile_path = os.path.join(data_dload_dir, tname)
    os.makedirs(S1tile_path, exist_ok=True)
    sentinel1_download_par(i,g,pol,name,tname,S1tile_path,scale,gpkg_files)
    
    

gfile /home/ljp238/Downloads/DRC_LiDAR_Roy2021/GEEdata/patches/S003E018/S003E018_tindex.gpkg
0/225 @S003E018_tindex :: 04
1/225 @S003E018_tindex :: 04
2/225 @S003E018_tindex :: 04


/tmp/ipykernel_3275336/2733603735.py:24: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  bBox = [float(ig.minx), float(ig.miny), float(ig.maxx), float(ig.maxy)]


['VV', 'VH', 'angle']
['VV', 'VH', 'angle']
['VV', 'VH', 'angle']
['VV', 'VH']
/media/ljp238/12TBWolf/ARCHIEVE/GEEDataDownload/S1_VVVH/S003E018_tindex/tile_1_S1_VVVH.tif
Generating URL ...
['VV', 'VH']
/media/ljp238/12TBWolf/ARCHIEVE/GEEDataDownload/S1_VVVH/S003E018_tindex/tile_0_S1_VVVH.tif
Generating URL ...
['VV', 'VH']
/media/ljp238/12TBWolf/ARCHIEVE/GEEDataDownload/S1_VVVH/S003E018_tindex/tile_10_S1_VVVH.tif
Generating URL ...
Please wait ...
Please wait ...
Please wait ...
Data downloaded to /media/ljp238/12TBWolf/ARCHIEVE/GEEDataDownload/S1_VVVH/S003E018_tindex/tile_1_S1_VVVH.tif
Data downloaded to /media/ljp238/12TBWolf/ARCHIEVE/GEEDataDownload/S1_VVVH/S003E018_tindex/tile_0_S1_VVVH.tif
Data downloaded to /media/ljp238/12TBWolf/ARCHIEVE/GEEDataDownload/S1_VVVH/S003E018_tindex/tile_10_S1_VVVH.tif


In [13]:
region, fname = geed.get_ee_geometry(i, g,name)
s1img = s1_median_image(region, pol)

/home/ljp238/Documents/UoE/dataops/dataops/GoogleEarthEngineDatasets.py:50: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  bBox = [float(ig.minx), float(ig.miny), float(ig.maxx), float(ig.maxy)]


['VV', 'VH', 'angle']
['VV', 'VH']
